In [1]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
x = torch.randn(2, 4)
net(x)

tensor([[0.1026],
        [0.2560]], grad_fn=<AddmmBackward>)

In [2]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[-0.2463,  0.1331,  0.0854,  0.3286, -0.1736, -0.1401,  0.0085,  0.0750]])), ('bias', tensor([0.1254]))])


In [3]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.1254], requires_grad=True)
tensor([0.1254])


In [4]:
net[0].weight.grad == None

True

In [5]:
# parameters only for the first layer
print(net[0].state_dict())
# parameters of the entire network
print(net.state_dict())

OrderedDict([('weight', tensor([[-0.0361,  0.4743,  0.2479, -0.0250],
        [-0.4142, -0.1750, -0.3655, -0.0489],
        [-0.3259,  0.4776,  0.4030,  0.1075],
        [-0.3833,  0.2880,  0.2671,  0.1117],
        [ 0.3833,  0.0776,  0.2841, -0.1086],
        [ 0.0166, -0.0705, -0.1538,  0.3901],
        [ 0.1596,  0.1052,  0.1528, -0.3148],
        [-0.3958, -0.0730,  0.4796, -0.0929]])), ('bias', tensor([ 0.4232, -0.3991, -0.4424,  0.3359,  0.0816, -0.0917,  0.4837, -0.3605]))])
OrderedDict([('0.weight', tensor([[-0.0361,  0.4743,  0.2479, -0.0250],
        [-0.4142, -0.1750, -0.3655, -0.0489],
        [-0.3259,  0.4776,  0.4030,  0.1075],
        [-0.3833,  0.2880,  0.2671,  0.1117],
        [ 0.3833,  0.0776,  0.2841, -0.1086],
        [ 0.0166, -0.0705, -0.1538,  0.3901],
        [ 0.1596,  0.1052,  0.1528, -0.3148],
        [-0.3958, -0.0730,  0.4796, -0.0929]])), ('0.bias', tensor([ 0.4232, -0.3991, -0.4424,  0.3359,  0.0816, -0.0917,  0.4837, -0.3605])), ('2.weight', tensor([

In [6]:
net.state_dict()['2.bias'].data

tensor([0.1254])

In [7]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(x)

tensor([[0.0417],
        [0.0417]], grad_fn=<AddmmBackward>)

In [8]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(x)

tensor([[-0.4420],
        [-0.4412]], grad_fn=<AddmmBackward>)

In [9]:
rgnet[0][1][0].bias.data

tensor([ 0.2937, -0.1031, -0.0230,  0.4057, -0.1909, -0.1828, -0.2487, -0.4858])

In [10]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(net[0].weight, mean=0, std=0.01)
        nn.init.zeros_(net[0].bias)
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([ 0.0107, -0.0058, -0.0089,  0.0032]), tensor(0.))

In [11]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.constant_(net[0].weight, 1)
        nn.init.zeros_(net[0].bias)
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [12]:
def xavier(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        torch.nn.init.constant_(m.weight, 42)

net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([-0.6599,  0.1358,  0.4828,  0.5439])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [13]:
def my_init(m):
    if type(m) == nn.Linear:
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[0:2]

tensor([[-0.0000, -0.0000,  7.5924,  0.0000],
        [-5.6645, -0.0000, -8.7134, -7.9989]], grad_fn=<SliceBackward>)

In [14]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000,  1.0000,  8.5924,  1.0000])

In [15]:
# We need to give the shared layer a name such that we can reference its
# parameters
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(x)
# Check whether the parameters are the same
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# Make sure that they are actually the same object rather than just having the
# same value
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])
